In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
from pyspark.sql import SQLContext
sc.stop()
sc =SparkContext()
sqlContext = SQLContext(sc)
import pyspark.sql.functions as func
import sys
from pyspark.sql.functions import countDistinct

In [2]:
# files = !gsutil ls gs://ds-url-catag/Stickers/sticker_bycategory/

In [3]:
# import re 
# count=0
# file_count=0
# for sub_file in files:
#     sub_str = sub_file.split("/")[5]
#     string = sub_str[13:]
#     category =  re.sub(r'\d','', string)
#     if sub_str:
#         file_count+=1
#         print file_count
#         try:
#             detail_catag = sc.textFile("gs://ds-url-catag/Stickers/sticker_bycategory/"+str(sub_str)).map(lambda line: line.split(",")).map(lambda line: (line[0],line[1],line[2])).toDF(['id','Name','animated'])
#             detail_catag = detail_catag.withColumn('category',lit(category))
#             print category
#             detail_catag.coalesce(1).write.parquet("gs://ds-url-catag/Stickers/proces_sticker_bycategory/"+str(category)+"/"+str(file_count)+"/")
#         except Exception,e:
#             count += 1 




In [8]:
Stick_category_data = sqlContext.read.parquet("gs://ds-url-catag/Stickers/proces_sticker_bycategory/*/*")
#Cleaning the data    
from pyspark.sql.types import StringType
import re
remove_quotes = udf(lambda x: re.sub('"','',x), StringType())
remove_slashes = udf(lambda x: re.sub('\\\\','',x), StringType())
#removing quotes from the data 
Stick_category_data_df = Stick_category_data.select(*[remove_quotes(column).alias(column) for column in Stick_category_data.columns])
Stick_category_data_df =  Stick_category_data_df.select(*[remove_slashes(column).alias(column) for column in Stick_category_data_df.columns])
Stick_category_data_df = Stick_category_data_df.dropDuplicates()

In [9]:
Stick_category_data_df.show()

+-------------------+--------------------+--------+----------------+
|                 id|                Name|animated|        category|
+-------------------+--------------------+--------+----------------+
|        cuteballoon|        CUTE BALLOON|    true|animatedstickers|
|           petspeak|From Frank Pet Speak|   false|  animalsandpets|
|   berthasvalentine|Bertha's Lovely V...|    true|animatedstickers|
|cottonanimalcostume|Cotton's Animal C...|   false|       bbmfamily|
|bbmfamilyvalentines|BBM Family Valent...|    true|animatedstickers|
|     royalbreakfast|     Royal Breakfast|   false|            food|
|      tontonfriends|      Tonton Friends|    true|animatedstickers|
|          chickyboy|          Chicky Boy|   false|  animalsandpets|
|        roscoracoon|       Rosco Raccoon|   false|  animalsandpets|
|   puppylovestochat| Puppy Loves to Chat|   false|  animalsandpets|
|    squarespression|      Squarepression|   false|           emoji|
|    preciousmoments|Precious Mome